In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
# 지표를 하나만 설정할 경우
from sklearn.model_selection import cross_val_score
# 지표를 하나 이상 설정할 경우
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 모델의 최적의 하이퍼파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 군집화
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth

# 시간 측정을 위한 시간 모듈
import datetime
# 주식 정보를 읽어오기 위한 라이브러리
from pandas_datareader import data

# 데이터 수집
import requests
from bs4 import BeautifulSoup
import re
import time
import os
import json

# 한국어 형태소 분석
from konlpy.tag import Okt, Hannanum, Kkma, Mecab, Komoran

# 워드 클라우드를 위한 라이브러리
from collections import Counter
import pytagcloud
from IPython.display import Image

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


### 데이터 수집

In [2]:
# 요청주소
site = 'https://openapi.naver.com/v1/search/news.json'

# 요청 해더
headers = {
    'X-Naver-Client-Id' : 'FulNrLNr3Ko3hCsCvf6V',
    'X-Naver-Client-Secret' : '0NojB3FwPR'
}

# 요청 파라미터
params = {
    'query' : '손흥민',
    'display' : 100,
    'start' : 1,
    'sort' : 'sim'
}

In [3]:
# 요청하는 함수
def getJson(site, params, headers) :
    res = requests.get(site, params=params, headers=headers)
    # json 객체로 생성한다.
    json_root = json.loads(res.text)
    return json_root

In [4]:
# 페이지 하나의 정보를 수집하는 함수
def getData(json_root) :
    # items를 추출힌다.
    items = json_root['items']
    
    # 데이터를 저장할 딕셔너리
    data_dict = {
        'title' : [],
        'description' : []
    }
    
    # items 안에 있는 딕셔너리의 수 만큼 반복한다.
    for item in items :
        # title을 가져온다.
        title = item['title']
        # description을 가져온다.
        description = item['description']
        
        # <b>와 <b/>를 제거한다.
        title = title.replace('<b>', '')
        title = title.replace('</b>', '')
        title = title.replace('&quot;', '')
        
        description = description.replace('<b>', '')
        description = description.replace('</b>', '')
        description = description.replace('&quot;', '')
        
        # print(title)
        # print(description)
        # print('---------------------------')
        
        # 딕셔너리에 담는다.
        data_dict['title'].append(title)
        data_dict['description'].append(description)
        
    # 수집한 데이터를 저장한다.
    df1 = pd.DataFrame(data_dict)
    
    return df1

In [5]:
# 전에 생성된 파일을 제거한다.
if os.path.isfile('data/son_log.txt') :
    os.remove('data/son_log.txt')

if os.path.isfile('data/son_data.csv') :
    os.remove('data/son_data.csv')
    
# 처음 저장여부를 확인하기 위한 변수
isSaved = False

while True :
    
    # 딜레이
    time.sleep(1)
    
    print(f'{params["start"]} 수집중')
    
    a1 = getJson(site, params, headers)
    df1 = getData(a1)

    # 요청한 주소를 저장한다.
    with open('data/son_log.txt', 'a', encoding='utf-8-sig') as fp:
        fp.write(str(params['start']) + '\n')

    if isSaved == False :
        # 처음 저장
        df1.to_csv('data/son_data.csv', encoding='utf-8-sig', index=False)
        isSaved = True
    else :
        df1.to_csv('data/son_data.csv', encoding='utf-8-sig', index=False, header=False, mode='a')
    
    # start 값을 계산한다.
    start = params['start'] + 100
    
    # 새롭게 구한 시작값이 1000보다 작으면 반복하게 한다.
    if start < 1000 :
        params['start'] = start
    else :
        # 그렇지  않으면 while문을 중단시킨다.
        print('수집완료')
        break


1 수집중
101 수집중
201 수집중
301 수집중
401 수집중
501 수집중
601 수집중
701 수집중
801 수집중
901 수집중
수집완료
